In [1]:
import json
import requests
import os
from datetime import datetime
import zipfile
import tempfile
from collections import defaultdict
import time

In [2]:
JSON_FILE = "./memories_history.json"

In [3]:
# Output folder
output_dir = "snapchat_downloads_renamed"
os.makedirs(output_dir, exist_ok=True)

In [4]:
# Load JSON
with open(JSON_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

In [5]:
memories = data['Saved Media']

In [6]:
def extension_from_headers(headers, media_type):
    content_type = headers.get("Content-Type", "").lower()

    if "video/mp4" in content_type:
        return ".mp4"
    if "image/jpeg" in content_type:
        return ".jpg"
    if "image/png" in content_type:
        return ".png"
    if "image/webp" in content_type:
        return ".webp"
    if "application/zip" in content_type:
        return ".zip"

    # fallback if Snapchat does something weird
    if media_type == "Video":
        return ".mp4"
    if media_type == "Image":
        return ".jpg"

    return ""
    
def safe_filename(s):
    return "".join(c for c in s if c.isalnum() or c in "._- ").rstrip()

def download_file(url, out_path, cookies=None):
    r = requests.get(url, stream=True, cookies=cookies)
    r.raise_for_status()
    with open(out_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return r.headers.get("content-type")

In [7]:
existing_files = []
for file in os.listdir("./snapchat_downloads_renamed"):
    existing_files.append(file.split(".")[0])

In [10]:
len(memories)

777

In [22]:
# Check for duplicate dates to ensure no memory will be overwritten
dates = []
for item in memories:
    dates.append(item["Date"])

dates_dict = {}
for date in dates:
    if date not in dates_dict:
        dates_dict[date]=1
    else:
        dates_dict[date]+=1
        print(date)



2025-08-17 18:42:39 UTC


In [8]:
for item in memories:
    url = item["Media Download Url"]
    media_type = item["Media Type"]

    # clean date for filename
    date = datetime.strptime(item["Date"], "%Y-%m-%d %H:%M:%S UTC")
    base_name = date.strftime("%Y-%m-%d_%H%M%S")
    
    # not downloading the same files we already downloaded before failure
    if base_name in existing_files:
        continue

    try:
        response = requests.get(url, allow_redirects=True)
        response.raise_for_status()
    except:
        if response.status_code >= 500:
            time.sleep(5)
            try:
                response = requests.get(url, allow_redirects=True)
                response.raise_for_status()
            except:
                print("Could not save: ", filename)
                continue
            

    ext = extension_from_headers(response.headers, media_type)
    filename = os.path.join(output_dir, base_name + ext)
    
    with open(filename, "wb") as f:
        f.write(response.content)

    print("Saved:", filename)


Saved: snapchat_downloads_renamed\2025-07-16_210344.zip
Saved: snapchat_downloads_renamed\2025-07-16_092252.jpg
Saved: snapchat_downloads_renamed\2025-07-15_151524.mp4
Saved: snapchat_downloads_renamed\2025-07-15_151434.jpg
Saved: snapchat_downloads_renamed\2025-07-15_151314.jpg
Saved: snapchat_downloads_renamed\2025-07-15_151303.jpg
Saved: snapchat_downloads_renamed\2025-07-15_151257.jpg
Saved: snapchat_downloads_renamed\2025-07-15_151248.jpg
Saved: snapchat_downloads_renamed\2025-07-15_151223.mp4
Saved: snapchat_downloads_renamed\2025-07-15_151111.mp4
Saved: snapchat_downloads_renamed\2025-07-15_151106.jpg
Saved: snapchat_downloads_renamed\2025-07-15_151035.mp4
Saved: snapchat_downloads_renamed\2025-07-15_150544.mp4
Saved: snapchat_downloads_renamed\2025-07-15_150506.mp4
Saved: snapchat_downloads_renamed\2025-07-15_150356.jpg
Saved: snapchat_downloads_renamed\2025-07-15_145945.mp4
Saved: snapchat_downloads_renamed\2025-07-15_145844.jpg
Saved: snapchat_downloads_renamed\2025-07-15_145

Saved: snapchat_downloads_renamed\2025-06-15_054526.jpg
Saved: snapchat_downloads_renamed\2025-06-15_043404.jpg
Saved: snapchat_downloads_renamed\2025-06-14_161939.jpg
Saved: snapchat_downloads_renamed\2025-06-14_120113.jpg
Saved: snapchat_downloads_renamed\2025-06-14_114456.jpg
Saved: snapchat_downloads_renamed\2025-06-14_074330.mp4
Saved: snapchat_downloads_renamed\2025-06-14_072612.mp4
Saved: snapchat_downloads_renamed\2025-06-14_072144.jpg
Saved: snapchat_downloads_renamed\2025-06-13_222818.jpg
Saved: snapchat_downloads_renamed\2025-06-13_001530.mp4
Saved: snapchat_downloads_renamed\2025-06-12_225038.jpg
Saved: snapchat_downloads_renamed\2025-06-12_202430.jpg
Saved: snapchat_downloads_renamed\2025-06-12_202413.jpg
Saved: snapchat_downloads_renamed\2025-06-12_132340.mp4
Saved: snapchat_downloads_renamed\2025-06-12_040440.jpg
Saved: snapchat_downloads_renamed\2025-06-12_035501.jpg
Saved: snapchat_downloads_renamed\2025-06-11_124330.jpg
Saved: snapchat_downloads_renamed\2025-06-11_123

Saved: snapchat_downloads_renamed\2025-05-14_072731.mp4
Saved: snapchat_downloads_renamed\2025-05-13_113455.jpg
Saved: snapchat_downloads_renamed\2025-05-13_070700.mp4
Saved: snapchat_downloads_renamed\2025-05-13_055259.mp4
Saved: snapchat_downloads_renamed\2025-05-13_055122.jpg
Saved: snapchat_downloads_renamed\2025-05-13_052257.mp4
Saved: snapchat_downloads_renamed\2025-05-13_050716.jpg
Saved: snapchat_downloads_renamed\2025-05-13_050642.zip
Saved: snapchat_downloads_renamed\2025-05-13_050406.jpg
Saved: snapchat_downloads_renamed\2025-05-13_050059.jpg
Saved: snapchat_downloads_renamed\2025-05-12_063244.jpg
Saved: snapchat_downloads_renamed\2025-05-12_041148.zip
Saved: snapchat_downloads_renamed\2025-05-12_023025.zip
Saved: snapchat_downloads_renamed\2025-05-09_104333.zip
Saved: snapchat_downloads_renamed\2025-05-09_104022.zip
Saved: snapchat_downloads_renamed\2025-05-09_035835.zip
Saved: snapchat_downloads_renamed\2025-05-07_011254.mp4
Saved: snapchat_downloads_renamed\2025-05-07_010

Saved: snapchat_downloads_renamed\2025-01-22_104928.jpg
Saved: snapchat_downloads_renamed\2025-01-22_104857.jpg
Saved: snapchat_downloads_renamed\2025-01-22_093503.jpg
Saved: snapchat_downloads_renamed\2025-01-22_091400.jpg
Saved: snapchat_downloads_renamed\2025-01-21_090013.jpg
Saved: snapchat_downloads_renamed\2025-01-20_145828.zip
Saved: snapchat_downloads_renamed\2025-01-20_093347.jpg
Saved: snapchat_downloads_renamed\2025-01-20_093336.jpg
Saved: snapchat_downloads_renamed\2025-01-20_090334.mp4
Saved: snapchat_downloads_renamed\2025-01-20_041110.jpg
Saved: snapchat_downloads_renamed\2025-01-19_013406.mp4
Saved: snapchat_downloads_renamed\2025-01-19_012906.mp4
Saved: snapchat_downloads_renamed\2025-01-18_151845.jpg
Saved: snapchat_downloads_renamed\2025-01-17_093218.jpg
Saved: snapchat_downloads_renamed\2025-01-17_060308.jpg
Saved: snapchat_downloads_renamed\2025-01-17_044400.mp4
Saved: snapchat_downloads_renamed\2025-01-15_093345.jpg
Saved: snapchat_downloads_renamed\2025-01-14_022

In [ ]:
# to do

# check for duplicates at start of file (maybe computationally expensive? rather than going one by one.. 
# compare doing it within the for loop vs ahead of it)

# see how complicated it would be to extract a zip file and label them _overlay or base or something 
# or see how many files are in each zip file and investigate if there's ever more than 2


In [27]:
url = "https://us-east1-aws.api.snapchat.com/dmd/mm?uid=3a28fa43-6906-47be-b3cc-cc7a5dacfb67&sid=59069641-9be7-4649-a9ac-42d1b833e65c&mid=59069641-9be7-4649-a9ac-42d1b833e65c&ts=1768656794529&sig=9f99cf5123fcde88bcc84cc02b0ed8d8618f4e6827be1dda47f7e8425afe28c7"
media_type = "Image"

# clean date for filename
date = datetime.strptime(item["Date"], "%Y-%m-%d %H:%M:%S UTC")
base_name = date.strftime("%Y-%m-%d_%H%M%S")

# not downloading the same files we already downloaded before failure

try:
    response = requests.get(url, allow_redirects=True)
    response.raise_for_status()
except:
    if response.status_code >= 500:
        time.sleep(5)
        try:
            response = requests.get(url, allow_redirects=True)
            response.raise_for_status()
        except:
            print("Could not save: ", filename)


ext = extension_from_headers(response.headers, media_type)
filename = os.path.join(output_dir, base_name + "_0" + ext)

with open(filename, "wb") as f:
    f.write(response.content)

print("Saved:", filename)


Saved: snapchat_downloads_renamed\2025-01-03_111212_0.jpg
